# Análisis Exploratorio de los datos


Database Format
Date:	 Date of accident,  in the format - January 01, 2001
Time:	 Local time, in 24 hr. format unless otherwise specified
Airline/Op:	 Airline or operator of the aircraft
Flight #:	 Flight number assigned by the aircraft operator
Route:	 Complete or partial route flown prior to the accident
AC Type:	 Aircraft type
Reg:	 ICAO registration of the aircraft
cn / ln:	 Construction or serial number / Line or fuselage number
Aboard:	 Total aboard (passengers / crew)
Fatalities:	 Total fatalities aboard (passengers / crew)
Ground:	 Total killed on the ground
Summary:	 Brief description of the accident and cause if known

In [227]:
import os
import pandas as pd

El dataset con el que se va a trabajar contiene información acerca de accidentes aéreos.

In [228]:
df = pd.read_csv(os.path.join(os.getcwd(),'data','AccidentesAviones.csv'))

In [229]:
df.head()

,Unnamed: 0,fecha,HORA declarada,Ruta,OperadOR,flight_no,route,ac_type,registration,cn_ln,all_aboard,PASAJEROS A BORDO,crew_aboard,cantidad de fallecidos,passenger_fatalities,crew_fatalities,ground,summary
0,0,"September 17, 1908",1718,"Fort Myer, Virginia",Military - U.S. Army,?,Demonstration,Wright Flyer III,?,1,2,1,1,1,1,0,0,"During a demonstration flight, a U.S. Army fly..."
1,1,"September 07, 1909",?,"Juvisy-sur-Orge, France",?,?,Air show,Wright Byplane,SC1,?,1,0,1,1,0,0,0,Eugene Lefebvre was the first pilot to ever be...
2,2,"July 12, 1912",0630,"Atlantic City, New Jersey",Military - U.S. Navy,?,Test flight,Dirigible,?,?,5,0,5,5,0,5,0,First U.S. dirigible Akron exploded just offsh...
3,3,"August 06, 1913",?,"Victoria, British Columbia, Canada",Private,?,?,Curtiss seaplane,?,?,1,0,1,1,0,1,0,The first fatal airplane accident in Canada oc...
4,4,"September 09, 1913",1830,Over the North Sea,Military - German Navy,?,?,Zeppelin L-1 (airship),?,?,20,?,?,14,?,?,0,The airship flew into a thunderstorm and encou...


Se reemplaza '?' por 'nan' para luego tener un registro de valores nulos.

In [230]:
df.replace('?',float('nan'),inplace=True)

In [231]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5008 entries, 0 to 5007
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Unnamed: 0              5008 non-null   int64 
 1   fecha                   5008 non-null   object
 2   HORA declarada          3504 non-null   object
 3   Ruta                    5003 non-null   object
 4   OperadOR                4998 non-null   object
 5   flight_no               1326 non-null   object
 6   route                   4246 non-null   object
 7   ac_type                 4995 non-null   object
 8   registration            4736 non-null   object
 9   cn_ln                   4341 non-null   object
 10  all_aboard              4991 non-null   object
 11  PASAJEROS A BORDO       4787 non-null   object
 12  crew_aboard             4789 non-null   object
 13  cantidad de fallecidos  5000 non-null   object
 14  passenger_fatalities    4773 non-null   object
 15  crew

### Unnamed: 0

La columna 'Unnamed: 0' no es de interés ya que es igual al índice.

In [232]:
len(df['Unnamed: 0'].unique())

5008

In [233]:
df.drop('Unnamed: 0',axis=1,inplace=True)

### Fecha

Es la fecha en la que ocurrió el accidente

Se convierte la columna fecha a formato datetime.

In [234]:
df.fecha = pd.to_datetime(df.fecha)

In [235]:
df.fecha = df.fecha.dt.strftime('%m/%d/%Y')

In [236]:
df.rename(columns={'fecha':'date'},inplace=True)

### HORA declarada

Hora en que ocurrió el accidente.

In [237]:
df.rename(columns={'HORA declarada':'time'},inplace=True)

In [238]:
df.time.fillna('-1',inplace=True)

In [239]:
def normalize_time(x):
    x = x.strip(' cZ')
    x = x.split(':')
    x = ''.join(x)
    x = x.split(';')
    return ''.join(x)

In [240]:
df.time = df.time.astype(str).apply(lambda x: int(normalize_time(x)))

### Ruta

Lugar en el que ocurrió el accidente.

In [241]:
df.rename(columns={'Ruta':'location'},inplace=True)

### OperadOR

Operador de la aerolinea.

In [242]:
df.rename(columns={'OperadOR':'operator'},inplace=True)

### flight_no

Número de vuelo. No se va a trabajar con esta columna ya que contiene casi el 75 % de valores nulos.

In [243]:
df.drop('flight_no',axis=1,inplace=True)

### route

Ruta de vuelo total o parcial antes del accidente.<br>

### ac_type 

Tipo de avión.

In [244]:
df.groupby('ac_type').count().nlargest(n=10,columns='date')

,date,time,location,operator,route,registration,cn_ln,all_aboard,PASAJEROS A BORDO,crew_aboard,cantidad de fallecidos,passenger_fatalities,crew_fatalities,ground,summary
ac_type,,,,,,,,,,,,,,,
Douglas DC-3,333,333,332,333,291,316,293,330,316,316,333,315,314,329,328
de Havilland Canada DHC-6 Twin Otter 300,81,81,81,81,68,81,81,81,81,81,81,81,81,81,81
Douglas C-47A,70,70,70,70,53,69,68,70,69,69,70,69,69,70,70
Douglas C-47,64,64,64,64,50,48,24,63,59,59,63,58,57,62,62
Douglas DC-4,41,41,41,41,38,40,41,41,40,40,41,40,40,40,41
Antonov AN-26,35,35,35,35,26,25,21,35,32,32,35,31,31,35,35
Yakovlev YAK-40,35,35,35,35,33,35,35,35,35,35,35,35,35,35,35
Junkers JU-52/3m,30,30,30,30,24,30,29,30,30,30,30,29,29,30,28
De Havilland DH-4,27,27,27,27,2,25,1,27,27,27,27,27,27,27,26


Los tipos de aviones que tuvieron más accidentes no tienen un número significativo.<br> Por tal motivo no se va a utilizar esta columna para el análisis.

In [245]:
df.drop('ac_type',axis=1,inplace=True)

### registration<br>
### cn_ln

registration y cn_ln son características de los aviones que tampoco se van a utilizar.

In [246]:
df.drop(['registration','cn_ln'],axis=1,inplace=True)

### all_aboard<br>
### PASAJEROS A BORDO<br>
### crew_aboard 

all_aboard es el número total de personas abordo y crew_aboard el total de la tripulación.

Se va a utilizar la columna del número total de personas. Ésta contiene valores nulos así que se la va a intentar llenar con la suma de valores de la columna ***PASAJEROS A BORDO*** y ***crew_aboard***.

In [247]:
df.crew_aboard = df.crew_aboard.apply(lambda x: float(x))

In [248]:
df['PASAJEROS A BORDO'] = df['PASAJEROS A BORDO'].apply(lambda x: float(x))

In [249]:
df.all_aboard = df.all_aboard.apply(lambda x: float(x))

In [250]:
df.all_aboard.fillna(df.crew_aboard + df['PASAJEROS A BORDO'],inplace=True)

In [251]:
df.drop(['PASAJEROS A BORDO','crew_aboard'],axis=1,inplace=True)

### cantidad de fallecidos<br>
### passenger_fatalities<br>
### crew_fatalities<br>
### ground 

La cantidad de fallecidos son los totales, passenger_fatalities son los pasajeros fallecidos, crew_fatalities el número de miembros de la flota fallecidos y ground los fallecidos en el suelo.<br>
Se va a trabajar con el número total.

In [252]:
df['cantidad de fallecidos'] = df['cantidad de fallecidos'].apply(lambda x: float(x))
df.passenger_fatalities = df.passenger_fatalities.apply(lambda x: float(x))
df.crew_fatalities = df.crew_fatalities.apply(lambda x: float(x))
df.ground = df.ground.apply(lambda x: float(x))

In [253]:
df['cantidad de fallecidos'].fillna(df.passenger_fatalities + df.crew_fatalities,inplace=True)

In [254]:
df['cantidad de fallecidos'].fillna(df.ground,inplace=True)

In [255]:
df.drop(['passenger_fatalities','crew_fatalities','ground'],axis = 1, inplace=True)

In [256]:
df.rename(columns={'cantidad de fallecidos':'total_fatalities'},inplace=True)

In [257]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5008 entries, 0 to 5007
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   date              5008 non-null   object 
 1   time              5008 non-null   int64  
 2   location          5003 non-null   object 
 3   operator          4998 non-null   object 
 4   route             4246 non-null   object 
 5   all_aboard        4991 non-null   float64
 6   total_fatalities  5000 non-null   float64
 7   summary           4949 non-null   object 
dtypes: float64(2), int64(1), object(5)
memory usage: 313.1+ KB


Se obtuvo información sobre la principal causa de algunos de los accidentes (ver ***more_info.ipynb***).

In [258]:
more_info = pd.read_csv(os.path.join(os.getcwd(),'data','more_info.csv'))

In [259]:
df2 = pd.merge(df, more_info, how = 'left', on=['date','location','operator'])

In [260]:
df2.drop(['Unnamed: 0','description'],axis=1,inplace=True)

In [261]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5008 entries, 0 to 5007
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   date              5008 non-null   object 
 1   time              5008 non-null   int64  
 2   location          5003 non-null   object 
 3   operator          4998 non-null   object 
 4   route             4246 non-null   object 
 5   all_aboard        4991 non-null   float64
 6   total_fatalities  5000 non-null   float64
 7   summary           4949 non-null   object 
 8   cause             66 non-null     object 
dtypes: float64(2), int64(1), object(6)
memory usage: 391.2+ KB


In [262]:
df2.cause.unique()

array([nan, 'Lightning', 'Cargo Hold / Cabin / Cockpit Fires',
       'Air Traffic Control Errors', 'Pilot Incapacitation',
       'Fuel Starvation', 'Sabotage / Hijacking', 'Design Flaws',
       'Bird Strikes'], dtype=object)

In [263]:
df2.cause = df2.cause.replace('Lightning','Weather')

### Summary

Se puede obtener más información de las causas a partir de la columna summary.

Se van a buscar palabras clave como ***weather***, ***air-traffic***, ***fuel***, etc.

In [264]:
import re

In [265]:
patterns = {
            'pilot mis': 'Pilot Incapacitation',
            "pilot's failure": 'Pilot Incapacitation',
            "pilot's error": 'Pilot Incapacitation',
            "pilot error": 'Pilot Incapacitation',
            'turbulence': 'Weather',
            'storm': 'Weather',
            'lightning': 'Weather',
            'fog': 'Weather',
            'visibility': 'Weather',
            'fire':'Cargo Hold / Cabin / Cockpit Fires',
            'burn':'Cargo Hold / Cabin / Cockpit Fires',
            'flames': 'Cargo Hold / Cabin / Cockpit Fires',
            'air traffic': 'Air Traffic Control Errors',
            'fuel': 'Fuel Starvation',
            'sabotage': 'Sabotage / Hijacking',
            'hijack': 'Sabotage / Hijacking',
            'design': 'Design Flaws',
            'engine': 'Engine Failures',
            'stall': 'Engine Failures',
            'bird strike': 'Bird Strikes',
            'shot down': 'Shot Down',
            'unknown': 'Unknown'
            }

In [266]:
patterns.keys()

dict_keys(['pilot mis', "pilot's failure", "pilot's error", 'pilot error', 'turbulence', 'storm', 'lightning', 'fog', 'visibility', 'fire', 'burn', 'flames', 'air traffic', 'fuel', 'sabotage', 'hijack', 'design', 'engine', 'stall', 'bird strike', 'shot down', 'unknown'])

In [267]:
cause = []
for i in df2.summary:
    cause_found=False
    for j in patterns.keys():
        if (not cause_found) and re.search(j,str(i).lower()):
            cause.append(patterns[re.search(j,str(i).lower()).group(0)])
            cause_found = True
    if not cause_found:
        cause.append(float('nan'))

In [268]:
df2['cause_2'] = cause

In [269]:
df2.cause.fillna(df2.cause_2,inplace=True)

In [270]:
df2.drop(['cause_2','summary'],axis=1,inplace=True)

In [271]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5008 entries, 0 to 5007
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   date              5008 non-null   object 
 1   time              5008 non-null   int64  
 2   location          5003 non-null   object 
 3   operator          4998 non-null   object 
 4   route             4246 non-null   object 
 5   all_aboard        4991 non-null   float64
 6   total_fatalities  5000 non-null   float64
 7   cause             2720 non-null   object 
dtypes: float64(2), int64(1), object(5)
memory usage: 352.1+ KB


Me interesan principalmente las filas que contienen datos del número de fallecidos y el número de total de personas en el avión.

In [272]:
df2.dropna(subset=['all_aboard','total_fatalities'],inplace=True)

In [273]:
df2.total_fatalities = df2.total_fatalities.apply(lambda x: int(x))

In [274]:
df2.all_aboard = df2.all_aboard.apply(lambda x: int(x))

In [275]:
df2.fillna('Sin Dato',inplace=True)

Con respecto a la información relacionada al lugar del accidente, se va a tomar sólo el país, o estado de USA.

In [276]:
df2.location = df2.location.apply(lambda x: x.split(" ")[-1].title())

In [277]:
df2.location = df2.location.apply(lambda x: x.split("\r")[-1])

In [278]:
df2.location = df2.location.apply(lambda x: x.split(",")[-1])

In [279]:
df2.location = df2.location.apply(lambda x: x.strip(',.)("2"'))

In [280]:
usa_states = ["Alabama",
"Alaska",
"Arizona",
"Arkansas",
"California",
'Calilfornia',
'Cailifornia',
'Californiia',
'Californiia',
"Ca",
"Colorado",
"Connecticut",
"Delaware",
"Florida",
"Georgia",
"Hawaii",
"Idaho",
"Illinois",
"Indiana",
"Iowa",
"Kansas",
"Kentucky",
"Louisiana",
"Maine",
"Maryland",
"Massachusetts",
"Michigan",
"Minnesota",
'Minnisota',
"Mississippi",
"Missouri",
"Montana",
"Nebraska",
"Nevada",
"New Hampshire",
"Hamshire",
"New Jersey",
"Jersey",
"New Mexico",
"New York",
"York",
"North Carolina",
"North Dakota",
"Ohio",
"Oklahoma",
"Oregon",
"Pennsylvania",
"Rhode Island",
"Island",
"South Carolina",
"Carolina",
"South Dakota",
"Dakota",
"Tennessee",
"Texas",
"Utah",
"Vermont",
"Virginia",
"Washington",
"West Virginia",
"Wisconsin",
'Wisconson',
"Wyoming",
"Wy",
"States",
"Dc",
"D.C",
"Ny",
"Usa",
'Tennesee',
'Deleware', 
'Massachutes',
'Airzona']

In [281]:
uk = ['Uk','England']

In [282]:
df2.location = df2.location.apply(lambda x: "USA" if x in usa_states else x)

In [283]:
df2.location = df2.location.apply(lambda x: "UK" if x in uk else x)

In [284]:
df2.location = df2.location.apply(lambda x: "Russia" if x=="Ussr" else x)

In [285]:
df2.location = df2.location.apply(lambda x: "Mexico" if x=="Mexic" else x)

In [286]:
df2.location = df2.location.apply(lambda x: "Australia" if x=="Australila" else x)

In [287]:
df2.drop(['route','operator'],inplace=True,axis=1)

In [288]:
df2.to_csv(os.path.join(os.getcwd(),'data','airplane_crashes.csv'),index=False)

### Carga de datos en MySQL

In [289]:
from common.create_database import create_database
from common.create_table import create_table
from common.load_data import load_data

In [290]:
create_database('accidentes_aereos')

La base de datos ha sido creada


In [291]:
variables_tabla = 'date VARCHAR(255),time INTEGER,location VARCHAR(255), all_aboard INTEGER, total_fatalities INTEGER, cause VARCHAR(255)'


In [292]:
create_table('accidentes_aereos','accidentes',variables_tabla)

Conectado a database:  ('accidentes_aereos',)
Creando tabla....
La tabla ha sido creada


In [293]:
data_route = os.path.join(os.getcwd(),'data','airplane_crashes.csv')

In [294]:
load_data(data_route,'accidentes_aereos','accidentes')

Conectado a database:  ('accidentes_aereos',)
Insertando datos...
